In [ ]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.chat_models import ChatOpenAI
from getpass import getpass
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from langchain.tools import tool

api_key = getpass("Enter your OpenAI API key: ")

In [ ]:
df = pd.read_csv("processed_data/crash_data.csv")
df.head()

,LATITUDE,LONGITUDE,ARC_ID,YEAR,VH,CYC,PED,FATAL,SERIOUS_INJURY,MINOR_INJURY,NO_INJURY,nearest_poi_type,distance_to_poi
0,34.022659,-118.438652,3815613985340226038495917846_38156133053402269...,2018.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,highway,0.007840
1,33.771053,-118.240303,3817593925337726257993907375_38175966053377030...,2018.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,highway,2.433526
2,34.025581,-118.335114,38166473753402558910272263152_3816648955340255...,2018.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,tourism,5.282972
3,34.164879,-118.410530,3815895015341648791916620220_38158843353416488...,2019.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,tourism,15.089772
4,33.969830,-118.347229,3816528095339698383331497996_38165261453396974...,2019.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,tourism,3.943337


In [3]:
@tool
def calculate_distance(query: str) -> str:
    """Calculate the geodesic distance in miles between two latitude/longitude coordinates."""
    return "distance"

csv_agent = create_csv_agent(
    ChatOpenAI(temperature=0, model="gpt-4o", api_key=api_key),
    path="processed_data/crash_data.csv",
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
    extra_tools=[calculate_distance],
)

/var/folders/dw/zncf0qf57rj7ky7vr3xsglw00000gn/T/ipykernel_12720/1831603582.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(temperature=0, model="gpt-4o", api_key=api_key),


In [4]:
response = csv_agent.invoke(str(input()))
print("Response:", response.get("output"))



> Entering new AgentExecutor chain...

Invoking: `calculate_distance` with `{'query': '34.025581 -118.335114, 34.0227 -118.439'}`


distance
Invoking: `calculate_distance` with `{'query': '34.025581 -118.335114, 33.7711 -118.24'}`


distance
Invoking: `calculate_distance` with `{'query': '34.025581 -118.335114, 34.0256 -118.335'}`


distance
Invoking: `calculate_distance` with `{'query': '34.025581 -118.335114, 34.1649 -118.411'}`


distance
Invoking: `calculate_distance` with `{'query': '34.025581 -118.335114, 33.9698 -118.347'}`


distanceThe nearest crash to the coordinates 34.025581, -118.335114 is at 34.0256, -118.335. Here is the information for that crash:

- **ARC_ID**: 38166473753402558910272263152_38166489553402558663036097
- **YEAR**: 2018
- **VH**: 0
- **CYC**: 0
- **PED**: 1
- **FATAL**: 0
- **SERIOUS_INJURY**: 0
- **MINOR_INJURY**: 0
- **NO_INJURY**: 1
- **nearest_poi_type**: tourism
- **distance_to_poi**: 5.28297 miles

> Finished chain.
Response: The nearest crash to 